In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
import datetime
print(datetime.datetime.now())

2019-10-25 01:14:05.206979


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


# Remove alll nan items and lower case the labels

In [6]:
D_ITEMS_df = pd.read_csv('D_ITEMS.csv')
print(D_ITEMS_df.shape)
D_ITEMS_df.head()

(12487, 10)


,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [8]:
D_ITEMS_df = D_ITEMS_df.dropna(subset=['LABEL'])
D_ITEMS_df['LABEL'] = D_ITEMS_df['LABEL'].str.lower()
D_ITEMS_df.head()

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,patient controlled analgesia (pca) [inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,pca lockout (min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,pca medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,pca total dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,pcv exh vt (obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN


# ICU Medications ITEMID

In [39]:
icu_medication = []
temp = D_ITEMS_df[D_ITEMS_df['LABEL'].str.contains('levophed')]
icu_medication = list(temp['ITEMID'])
temp

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
5794,5367,30047,levophed,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5839,5414,30120,levophed-k,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN


In [55]:
D_ITEMS_df[D_ITEMS_df['LABEL'].str.contains('adenosine')]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
1068,1569,4649,adenosine,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
8878,12959,221282,adenosine,Adenosine,metavision,inputevents_mv,Medications,mg,Solution,NaN


In [40]:
temp = D_ITEMS_df[D_ITEMS_df['LABEL'].str.contains('vasopressin')]
icu_medication.extend(temp['ITEMID'])
temp

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
124,1276,1136,vasopressin,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1185,1314,1222,vasopressin,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1212,1341,1327,vasopressin unit/min,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3079,3448,2248,vasopressin unit/min,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3116,3485,2334,vasopressin u/hr,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3188,3557,2445,vasopressin,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3269,3638,2561,vasopressin u/hr,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3403,3772,2765,vasopressin unit/r,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4081,4081,6255,vasopressin 0.04 s,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4337,4614,7341,vasopressin u/hr,NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [42]:
temp = D_ITEMS_df[D_ITEMS_df['LABEL'].str.contains('epinephrine')]
icu_medication.extend(list(temp['ITEMID']))
temp

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
1613,794,3112,epinephrine mcg/min,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4442,5133,30119,epinephrine-k,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5683,5227,30309,epinephrine drip,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
5792,5365,30044,epinephrine,NaN,carevue,inputevents_cv,NaN,NaN,NaN,NaN
8879,12960,221289,epinephrine,Epinephrine,metavision,inputevents_mv,Medications,mg,Solution,NaN
9553,12984,221906,norepinephrine,Norepinephrine,metavision,inputevents_mv,Medications,mg,Solution,NaN


In [43]:
icu_medication

[30047,
 30120,
 1136,
 1222,
 1327,
 2248,
 2334,
 2445,
 2561,
 2765,
 6255,
 7341,
 30051,
 42273,
 42802,
 222315,
 3112,
 30119,
 30309,
 30044,
 221289,
 221906]

# Diabetes Chartevents

In [8]:
print(datetime.datetime.now())
df = pd.read_csv('CHARTEVENTS.csv',chunksize=10**6)
chartevents_df =  pd.DataFrame()
go = True
while go:
    try:
        events_df = df.get_chunk()
        events_df = events_df[events_df['ITEMID'].isin([1455, 1310, 807, 3744, 3745, 2338, 226537])]
        events_df = events_df[events_df['VALUENUM']>=200]
        events_temp = events_df.drop_duplicates(subset=['SUBJECT_ID'])
        chartevents_df=chartevents_df.append(events_temp)
        chartevents_df = chartevents_df.drop_duplicates(subset=['SUBJECT_ID'])
        print(chartevents_df.shape)
    except Exception as e:
        print (type(e))
        go = False
print(chartevents_df.shape)
print(datetime.datetime.now())

2019-10-02 00:25:53.210223
(53, 15)
(78, 15)
(132, 15)
(247, 15)
(358, 15)
(399, 15)
(468, 15)
(529, 15)
(587, 15)
(641, 15)
(698, 15)
(750, 15)
(807, 15)
(827, 15)
(880, 15)
(909, 15)
(964, 15)
(1016, 15)
(1077, 15)
(1124, 15)
(1192, 15)
(1246, 15)
(1316, 15)
(1361, 15)
(1408, 15)
(1467, 15)
(1507, 15)
(1556, 15)
(1609, 15)
(1644, 15)
(1696, 15)
(1754, 15)
(1810, 15)
(1865, 15)
(1912, 15)
(1956, 15)
(1986, 15)
(2015, 15)
(2047, 15)
(2082, 15)
(2117, 15)
(2160, 15)
(2204, 15)
(2244, 15)
(2294, 15)
(2322, 15)
(2363, 15)
(2410, 15)
(2455, 15)
(2498, 15)
(2526, 15)
(2569, 15)
(2615, 15)
(2657, 15)
(2685, 15)
(2728, 15)
(2784, 15)
(2823, 15)
(2851, 15)
(2899, 15)
(2934, 15)
(2962, 15)
(2999, 15)
(3037, 15)
(3061, 15)
(3114, 15)
(3151, 15)
(3188, 15)
(3239, 15)
(3276, 15)
(3305, 15)
(3339, 15)
(3373, 15)
(3396, 15)
(3428, 15)
(3465, 15)
(3506, 15)
(3538, 15)
(3562, 15)
(3600, 15)
(3642, 15)
(3683, 15)
(3719, 15)
(3757, 15)
(3799, 15)
(3844, 15)
(3872, 15)
(3913, 15)
(3954, 15)
(3998, 15)
(4

c:\users\daniel chang\appdata\local\programs\python\python37\lib\site-packages\IPython\core\async_helpers.py:67: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)


c:\users\daniel chang\appdata\local\programs\python\python37\lib\site-packages\IPython\core\async_helpers.py:67: DtypeWarning: Columns (8,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)


c:\users\daniel chang\appdata\local\programs\python\python37\lib\site-packages\IPython\core\async_helpers.py:67: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
(8197, 15)
<class 'StopIteration'>
(8197, 15)
2019-10-02 00:32:13.385520


In [9]:
chartevents_df = chartevents_df.sort_values(by=['SUBJECT_ID'])
chartevents_df = chartevents_df.drop(['ROW_ID'], axis = 1)
chartevents_df

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
34061542,3,145834,211552.0,807,2101-10-21 06:00:00,2101-10-21 05:46:00,21570.0,306,306.0,NaN,NaN,NaN,Manual,NotStopd
34028522,4,185777,294638.0,807,2191-03-16 08:00:00,2191-03-16 07:49:00,17144.0,266,266.0,NaN,NaN,NaN,Manual,NotStopd
34297591,6,107064,228232.0,807,2175-05-30 21:00:00,2175-05-30 21:54:00,15443.0,254,254.0,NaN,NaN,NaN,Manual,NotStopd
34086330,9,150750,220597.0,807,2149-11-13 12:00:00,2149-11-13 19:10:00,20855.0,212,212.0,NaN,NaN,NaN,Manual,NotStopd
34062720,13,143045,263738.0,807,2167-01-08 23:30:00,2167-01-08 23:21:00,21205.0,205,205.0,NaN,NaN,NaN,Manual,NotStopd
34049478,18,188822,298129.0,807,2167-10-03 11:00:00,2167-10-03 11:01:00,19667.0,243,243.0,NaN,NaN,NaN,Manual,NotStopd
34035552,21,111970,216859.0,807,2135-02-04 06:00:00,2135-02-04 06:05:00,15907.0,200,200.0,NaN,NaN,NaN,Manual,NotStopd
34033889,25,129635,203487.0,807,2160-11-02 04:45:00,2160-11-02 05:23:00,18654.0,353,353.0,NaN,NaN,NaN,Manual,NotStopd
34406341,43,146828,225852.0,807,2186-10-04 07:00:00,2186-10-04 07:29:00,17435.0,210,210.0,NaN,NaN,NaN,Manual,NotStopd
34510579,56,181711,275642.0,807,2104-01-03 16:00:00,2104-01-03 16:19:00,21347.0,201,201.0,NaN,NaN,NaN,Manual,NotStopd


In [14]:
chartevents_df['SUBJECT_ID'].nunique()

8197

# Read Whole Dataset

In [6]:
print(datetime.datetime.now())
df = pd.read_csv('CHARTEVENTS.csv',chunksize=10**6)
chartevents_data_df =  pd.DataFrame()
# diabetes_subjectID = chartevents_df['SUBJECT_ID'].unique().tolist()
diabetes_subjectID = [109, 188, 209, 249, 3, 6]
go = True
while go:
    try:
        events_df = df.get_chunk()
        events_df = pd.merge(events_df, D_ITEMS_df, how='inner', on=['ITEMID'])
        events_df = events_df[['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','ITEMID','VALUENUM','LABEL']]
        events_temp = events_df.dropna(subset=['VALUENUM'])
#         events_temp = events_df[events_df['SUBJECT_ID'].isin(diabetes_subjectID)]
        chartevents_data_df=chartevents_data_df.append(events_temp)
        print(chartevents_data_df.shape)
    except Exception as e:
        print (type(e))
        go = False
print(chartevents_data_df.shape)
print(datetime.datetime.now())

2019-10-20 04:40:51.454365
(1000000, 6)
(2000000, 6)
(3000000, 6)
(4000000, 6)
(5000000, 6)
(6000000, 6)
(7000000, 6)
(8000000, 6)
(9000000, 6)
(10000000, 6)
(11000000, 6)
(12000000, 6)
(13000000, 6)
(14000000, 6)
(15000000, 6)
(16000000, 6)
(17000000, 6)
(18000000, 6)
(19000000, 6)
(20000000, 6)
(21000000, 6)
(22000000, 6)
(23000000, 6)
(24000000, 6)
(25000000, 6)
(26000000, 6)
(27000000, 6)
(28000000, 6)
(29000000, 6)
(30000000, 6)
(31000000, 6)
(32000000, 6)
(33000000, 6)
(34000000, 6)
(34599656, 6)
(34995097, 6)
(35414238, 6)
(35835707, 6)
(36237315, 6)
(36640179, 6)
(37048797, 6)
(37430869, 6)
(37837860, 6)
(38242095, 6)
(38643002, 6)
(39069824, 6)
(39478500, 6)
(39895723, 6)
(40292753, 6)
(40707096, 6)
(41113806, 6)
(41511090, 6)
(41917143, 6)
(42311072, 6)
(42718368, 6)
(43121820, 6)
(43526135, 6)
(43929651, 6)
(44351402, 6)
(44782302, 6)
(45185379, 6)
(45614412, 6)
(46032279, 6)
(46435392, 6)
(46838317, 6)
(47243702, 6)
(47639284, 6)
(48028907, 6)
(48423720, 6)
(48827184, 6)
(4

C:\Users\Daniel Chang\Anaconda3\lib\site-packages\IPython\core\async_helpers.py:68: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(98784673, 6)
(99295684, 6)
(99810936, 6)
(100309790, 6)
(100818147, 6)
(101334230, 6)
(101846441, 6)
(102351302, 6)
(102846824, 6)
(103357783, 6)
(103871134, 6)
(104383553, 6)
(104909110, 6)
(105420464, 6)
(105934774, 6)
(106449048, 6)
(106960830, 6)
(107464529, 6)
(107979832, 6)
(108493967, 6)
(109010833, 6)
(109513810, 6)
(110016293, 6)
(110512517, 6)
(111018502, 6)
(111520882, 6)
(112012364, 6)
(112561441, 6)
(113135907, 6)
(113714803, 6)
(114288512, 6)
(114858145, 6)
(115434274, 6)
(116016220, 6)
(116592762, 6)
(117170428, 6)
(117743887, 6)
(118329018, 6)
(118914483, 6)
(119484865, 6)
(120051403, 6)
(120629327, 6)
(121217119, 6)
(121805508, 6)
(122400479, 6)
(122983193, 6)
(123555382, 6)
(124140546, 6)
(124711655, 6)
(125290216, 6)
(125871148, 6)
(126445393, 6)
(127021270, 6)
(127588180, 6)
(128156120, 6)
(128725516, 6)
(129298868, 6)
(129886043, 6)
(130514973, 6)
(131151405, 6)
(131791199, 6)
(132431293, 6)


C:\Users\Daniel Chang\Anaconda3\lib\site-packages\IPython\core\async_helpers.py:68: DtypeWarning: Columns (8,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(133203186, 6)
(134203186, 6)
(135203186, 6)
(136203186, 6)
(137203185, 6)
(138203185, 6)


C:\Users\Daniel Chang\Anaconda3\lib\site-packages\IPython\core\async_helpers.py:68: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(138562041, 6)
(138687870, 6)
(138810787, 6)
(138931511, 6)
(139054412, 6)
(139184973, 6)
(139318411, 6)
(139451921, 6)
(139583226, 6)
(139704718, 6)
(139816632, 6)
(139930210, 6)
(140047816, 6)
(140160925, 6)
(140273459, 6)
(140386726, 6)
(140502178, 6)
(140616709, 6)
(140732798, 6)
(140843235, 6)
(140959902, 6)
(141080944, 6)
(141195099, 6)
(141307118, 6)
(141419207, 6)
(141527967, 6)
(141638197, 6)
(141750282, 6)
(141864508, 6)
(141976308, 6)
(142089028, 6)
(142204515, 6)
(142320527, 6)
(142436916, 6)
(142552443, 6)
(142664009, 6)
(142783692, 6)
(142897535, 6)
(143007668, 6)
(143120609, 6)
(143235462, 6)
(143348876, 6)
(143459076, 6)
(143576255, 6)
(143689432, 6)
(143806606, 6)
(143921348, 6)
(144033190, 6)
(144144777, 6)
(144255317, 6)
(144374820, 6)
(144490294, 6)
(144603833, 6)
(144719660, 6)
(144833565, 6)
(144948193, 6)
(145065215, 6)
(145180912, 6)
(145293420, 6)
(145411629, 6)
(145522980, 6)
(145637205, 6)
(145751663, 6)
(145863379, 6)
(145973804, 6)
(146088517, 6)
(146199043

# Read Medication

In [44]:
print(datetime.datetime.now())
df = pd.read_csv('CHARTEVENTS.csv',chunksize=10**6)
medication_df =  pd.DataFrame()
go = True
while go:
    try:
        events_df = df.get_chunk()
        events_df = pd.merge(events_df, D_ITEMS_df, how='inner', on=['ITEMID'])
        events_df = events_df[['SUBJECT_ID','ICUSTAY_ID','CHARTTIME','ITEMID','VALUENUM','LABEL']]
        events_temp = events_df.dropna(subset=['VALUENUM'])
        events_temp = events_temp[events_temp['ITEMID'].isin(icu_medication)]
        medication_df=medication_df.append(events_temp)
        print(medication_df.shape)
    except Exception as e:
        print (type(e))
        go = False
print(medication_df.shape)
print(datetime.datetime.now())

2019-10-25 01:33:53.003154
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(0, 6)
(3, 6)
(3, 6)
(3, 6)
(3, 6)
(4, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(8, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(19, 6)
(28, 6)
(28, 6)
(76, 6)
(76, 6)
(106, 6)
(106, 6)
(106, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(107, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)
(108, 6)


C:\Users\Daniel Chang\Anaconda3\lib\site-packages\IPython\core\async_helpers.py:68: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)


C:\Users\Daniel Chang\Anaconda3\lib\site-packages\IPython\core\async_helpers.py:68: DtypeWarning: Columns (8,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)


C:\Users\Daniel Chang\Anaconda3\lib\site-packages\IPython\core\async_helpers.py:68: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  coro.send(None)


(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
(208, 6)
<class 'StopIteration'>
(208, 6)
2019-10-25 01:46:19.767228


In [45]:
medication_df

,SUBJECT_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUENUM,LABEL
946069,3294,272416.0,2194-12-09 05:00:00,1136,3.0000,vasopressin
946070,3294,272416.0,2194-12-09 06:00:00,1136,3.0000,vasopressin
946071,3294,272416.0,2194-12-09 04:00:00,1136,3.0000,vasopressin
999988,4521,284189.0,2103-11-07 09:00:00,6255,0.0400,vasopressin 0.04 s
918782,4521,284189.0,2103-11-07 08:00:00,6255,0.0400,vasopressin 0.04 s
918783,4521,284189.0,2103-11-07 06:00:00,6255,0.0400,vasopressin 0.04 s
918784,4521,284189.0,2103-11-07 07:00:00,6255,0.0400,vasopressin 0.04 s
918785,4521,284189.0,2103-11-07 10:00:00,6255,0.0400,vasopressin 0.04 s
998740,8406,239553.0,2149-05-08 19:00:00,2248,0.0025,vasopressin unit/min
998741,8406,239553.0,2149-05-08 21:00:00,2248,0.0025,vasopressin unit/min


In [52]:
medication_df['LABEL'].value_counts()

vasopressin unit/min    88
vasopressin             77
epinephrine mcg/min     30
vasopressin  u/hr        7
vasopressin 0.04   s     5
vasopressin u/hr         1
Name: LABEL, dtype: int64

# Numbers of unique Labels

In [11]:
print (chartevents_data_df['SUBJECT_ID'].nunique())
print (chartevents_data_df['SUBJECT_ID'].unique())

6
[109 188 209 249   3   6]


# Check subject 6 dataframe

In [12]:
chartevents_6_df = chartevents_data_df[chartevents_data_df['SUBJECT_ID']==6]
chartevents_6_df.head()

,SUBJECT_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUENUM,LABEL
301523,6,228232.0,2175-06-03 00:58:00,861,4.9,"wbc (4-11,000)"
301535,6,228232.0,2175-05-31 01:48:00,861,10.6,"wbc (4-11,000)"
301543,6,228232.0,2175-06-02 15:18:00,861,2.9,"wbc (4-11,000)"
301544,6,228232.0,2175-06-02 02:06:00,861,8.5,"wbc (4-11,000)"
301555,6,228232.0,2175-06-01 03:00:00,861,10.2,"wbc (4-11,000)"


# Found useful labels

In [13]:
#spo2 = 646
chartevents_6_df[chartevents_6_df['LABEL'].str.contains('spo2')]
#heart rate = 211
chartevents_6_df[chartevents_6_df['LABEL'].str.contains('heart rate')]
#cvp = 113
chartevents_6_df[chartevents_6_df['LABEL'].str.match('cvp')]
#arterial bp mean = 52
chartevents_6_df[chartevents_6_df['LABEL'].str.match('arterial bp mean')]
# respiratory rate = 618
chartevents_6_df[chartevents_6_df['LABEL'].str.match('respiratory rate')]
#nbp mean = 456
chartevents_6_df[chartevents_6_df['ITEMID']==456]
#temp f = 678
chartevents_6_df[chartevents_6_df['LABEL'].str.match('temperature f')]

,SUBJECT_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUENUM,LABEL
530477,6,228232.0,2175-06-03 03:00:00,678,96.699997,temperature f
530495,6,228232.0,2175-05-31 02:00:00,678,97.099998,temperature f
530496,6,228232.0,2175-06-01 07:00:00,678,98.599998,temperature f
530519,6,228232.0,2175-06-03 11:00:00,678,96.500000,temperature f
530540,6,228232.0,2175-05-31 07:00:00,678,96.900002,temperature f
530548,6,228232.0,2175-06-02 03:00:00,678,99.099998,temperature f
530588,6,228232.0,2175-05-31 12:00:00,678,97.099998,temperature f
530589,6,228232.0,2175-05-31 12:30:00,678,96.800003,temperature f
530590,6,228232.0,2175-05-31 13:00:00,678,97.500000,temperature f
530642,6,228232.0,2175-06-02 15:00:00,678,98.099998,temperature f


In [14]:
#heart rate = 211
chartevents_3_df[chartevents_3_df['LABEL'].str.contains('heart rate')]

NameError: name 'chartevents_3_df' is not defined

In [35]:
#spo2 = 646
print(chartevents_3_df[chartevents_3_df['ITEMID']==646].shape)
      
#heart rate = 211
print(chartevents_3_df[chartevents_3_df['ITEMID']==211].shape)
      
#cvp = 113
print(chartevents_3_df[chartevents_3_df['ITEMID']==113].shape)
      
#arterial bp mean = 52
print(chartevents_3_df[chartevents_3_df['ITEMID']==52].shape)
      
#respiratory rate = 618
print(chartevents_3_df[chartevents_3_df['ITEMID']==618].shape)
      
#nbp mean = 456
print(chartevents_3_df[chartevents_3_df['ITEMID']==456].shape)
#temp f
print(chartevents_3_df[chartevents_3_df['ITEMID']==678].shape)

print(chartevents_3_df[chartevents_3_df['ITEMID']==678].shape)
print(chartevents_3_df[chartevents_3_df['ITEMID']==677].shape)
print(chartevents_3_df[chartevents_3_df['ITEMID']==674].shape)

(165, 6)
(170, 6)
(93, 6)
(112, 6)
(166, 6)
(64, 6)
(16, 6)
(16, 6)
(16, 6)
(17, 6)


# Functions that output attributes

In [46]:
def getFeatures(df,itemId, label):
    temp = df[df['ITEMID']==itemId]
    temp = temp.drop(['ITEMID','LABEL'], axis=1)
    temp = temp.rename(index=str, columns={"VALUENUM": label})
    return temp

In [47]:
def getItems(df):
    items_df = pd.DataFrame(columns=['ITEMID', 'LABEL'])
    for x in [646,52,618,8368,51,497,678]:
        temp = D_ITEMS_df[D_ITEMS_df['ITEMID']==x]
        temp = temp.drop(temp.columns.difference(['ITEMID','LABEL']),axis=1)
        items_df = pd.concat([items_df,temp], ignore_index=True)
    return items_df

In [48]:
print(datetime.datetime.now())
print(datetime.datetime.now()+ datetime.timedelta(hours=1))
print(datetime.datetime.now()- datetime.timedelta(hours=1))

2019-10-25 01:52:38.264961
2019-10-25 02:52:38.264961
2019-10-25 00:52:38.264961


In [49]:
def get_diabetes(df):
    diabetes_df = getFeatures(df,211,'heart rate')
    diabetes_items = getItems(df)
    diabetes_items = diabetes_items[diabetes_items['ITEMID'].isin([646,52,618,113,8368,51])]
    for index, col in diabetes_items.iterrows():
        temp = getFeatures(df,col['ITEMID'],col['LABEL'])
        diabetes_df = pd.merge(diabetes_df, temp, how='left', on=['SUBJECT_ID', 'ICUSTAY_ID','CHARTTIME'])
    return diabetes_df

In [50]:
def get_DataFrame(df):
    diabetes_df = getFeatures(df,211,'heart rate')
    diabetes_items = getItems(df)
    diabetes_items = diabetes_items[diabetes_items['ITEMID'].isin([646,52,618,113,8368,51])]
    for index, col in diabetes_items.iterrows():
        temp = getFeatures(df,col['ITEMID'],col['LABEL'])
        diabetes_df = pd.merge(diabetes_df, temp, how='outer', on=['SUBJECT_ID', 'ICUSTAY_ID','CHARTTIME'])
    return diabetes_df

In [12]:
def df_nan(df):
    print(df.shape)
    print(df.isna().sum())

# Filter out non-0 time from whole chartevents

In [14]:
print(datetime.datetime.now())
print(chartevents_data_df['SUBJECT_ID'].shape)
chartevents_all_df = chartevents_data_df[pd.to_datetime(chartevents_data_df['CHARTTIME']).dt.minute==0]
print(chartevents_all_df['SUBJECT_ID'].shape)
print(datetime.datetime.now())

2019-10-20 06:40:39.951021
(146504168,)
(102958880,)
2019-10-20 06:43:10.219318


In [21]:
chartevents_all_df

,SUBJECT_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUENUM,LABEL
0,36,241249.0,2134-05-12 12:00:00,223834,15.0,o2 flow
1,23,234044.0,2157-10-21 20:00:00,223834,3.0,o2 flow
2,23,234044.0,2157-10-21 22:00:00,223834,2.0,o2 flow
3,23,234044.0,2157-10-22 00:00:00,223834,2.0,o2 flow
5,36,241249.0,2134-05-12 10:00:00,223834,15.0,o2 flow
6,34,290505.0,2191-02-23 12:00:00,223834,3.0,o2 flow
7,34,290505.0,2191-02-24 00:00:00,223834,2.0,o2 flow
9,36,241249.0,2134-05-15 21:00:00,223834,15.0,o2 flow
10,36,241249.0,2134-05-16 00:00:00,223834,15.0,o2 flow
11,36,241249.0,2134-05-16 08:00:00,223834,4.0,o2 flow


# Data Processing

In [56]:
def align_dataframe(x,charteventsf):
    df = charteventsf[charteventsf['SUBJECT_ID']==x].sort_values(by=['CHARTTIME'])
    df = get_DataFrame(df)
    df = df.dropna()
    df = df[pd.to_datetime(df['CHARTTIME']).dt.minute==0]
    df['TARGET_hr'] = df['heart rate'].shift(periods=-1)
    df['TARGET_spo2'] = df['spo2'].shift(periods=-1)
    df['TARGET_map'] = df['arterial bp mean'].shift(periods=-1)
    df['TARGET_dbp'] = df['arterial bp [diastolic]'].shift(periods=-1)
    df['TARGET_sbp'] = df['arterial bp [systolic]'].shift(periods=-1)
    df['TARGET_resp'] = df['respiratory rate'].shift(periods=-1)
    df['TIME'] = df['CHARTTIME'].shift(periods=-1)
    df = df[pd.to_datetime(df['CHARTTIME']).dt.hour==
            (pd.to_datetime(df['TIME']).dt.hour-1)]
    return df

In [25]:
print(datetime.datetime.now())

# items to be removed 
test_subject = {109, 188, 209, 249, 3, 6} 
diabetes_subjectID = [ele for ele in diabetes_subjectID if ele not in test_subject]

icu_1hour_df = pd.DataFrame()

for x in chartevents_all_df['SUBJECT_ID'].unique().tolist():
    temp = align_dataframe(x,chartevents_all_df)
    icu_1hour_df = icu_1hour_df.append(temp, sort = False)

print(datetime.datetime.now())

2019-10-20 06:52:57.473270
2019-10-20 11:35:35.996832


In [26]:
icu_1hour_df.to_csv('Vital Sign Full Data 2.csv',index=0)

In [27]:
icu_1hour_df

,SUBJECT_ID,ICUSTAY_ID,CHARTTIME,heart rate,spo2,arterial bp mean,respiratory rate,arterial bp [diastolic],arterial bp [systolic],TARGET_hr,TARGET_spo2,TARGET_map,TARGET_dbp,TARGET_sbp,TARGET_resp,TIME
0,36,280987.0,2131-05-04 14:00:00,90.0,100.0,85.0,11.0,76.0,97.0,90.0,100.0,77.0,67.0,102.0,18.0,2131-05-04 15:00:00
1,36,280987.0,2131-05-04 15:00:00,90.0,100.0,77.0,18.0,67.0,102.0,90.0,100.0,83.0,71.0,112.0,18.0,2131-05-04 16:00:00
2,36,280987.0,2131-05-04 16:00:00,90.0,100.0,83.0,18.0,71.0,112.0,80.0,100.0,112.0,94.0,145.0,13.0,2131-05-04 17:00:00
3,36,280987.0,2131-05-04 17:00:00,80.0,100.0,112.0,13.0,94.0,145.0,71.0,100.0,87.0,70.0,122.0,17.0,2131-05-04 18:00:00
4,36,280987.0,2131-05-04 18:00:00,71.0,100.0,87.0,17.0,70.0,122.0,77.0,100.0,80.0,65.0,117.0,17.0,2131-05-04 19:00:00
5,36,280987.0,2131-05-04 19:00:00,77.0,100.0,80.0,17.0,65.0,117.0,84.0,99.0,83.0,67.0,115.0,19.0,2131-05-04 20:00:00
6,36,280987.0,2131-05-04 20:00:00,84.0,99.0,83.0,19.0,67.0,115.0,85.0,97.0,76.0,62.0,111.0,19.0,2131-05-04 21:00:00
7,36,280987.0,2131-05-04 21:00:00,85.0,97.0,76.0,19.0,62.0,111.0,88.0,98.0,71.0,57.0,102.0,19.0,2131-05-04 22:00:00
8,36,280987.0,2131-05-04 22:00:00,88.0,98.0,71.0,19.0,57.0,102.0,85.0,97.0,66.0,62.0,69.0,12.0,2131-05-04 23:00:00
29,36,211200.0,2131-05-17 07:00:00,85.0,100.0,73.0,19.0,60.0,99.0,70.0,96.0,86.0,71.0,114.0,20.0,2131-05-17 08:00:00


# Medication Col

In [57]:
medication_df.head()

,SUBJECT_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUENUM,LABEL
946069,3294,272416.0,2194-12-09 05:00:00,1136,3.00,vasopressin
946070,3294,272416.0,2194-12-09 06:00:00,1136,3.00,vasopressin
946071,3294,272416.0,2194-12-09 04:00:00,1136,3.00,vasopressin
999988,4521,284189.0,2103-11-07 09:00:00,6255,0.04,vasopressin 0.04 s
918782,4521,284189.0,2103-11-07 08:00:00,6255,0.04,vasopressin 0.04 s


In [60]:
getFeatures(medication_df,1136, '12')

,SUBJECT_ID,ICUSTAY_ID,CHARTTIME,12
946069,3294,272416.0,2194-12-09 05:00:00,3.00
946070,3294,272416.0,2194-12-09 06:00:00,3.00
946071,3294,272416.0,2194-12-09 04:00:00,3.00
996159,10568,295348.0,2119-04-13 05:00:00,0.04
996160,10568,295348.0,2119-04-13 04:00:00,0.04
996161,10568,295348.0,2119-04-13 02:00:00,0.04
997067,20086,272317.0,2199-02-11 14:00:00,0.04
997068,20086,272317.0,2199-02-11 15:00:00,0.04
997069,20086,272317.0,2199-02-11 00:00:00,0.04
997070,20086,272317.0,2199-02-10 19:00:00,0.04


In [ ]:
def getFeatures(df,itemId, label):
    temp = df[df['ITEMID']==itemId]
    temp = temp.drop(['ITEMID','LABEL'], axis=1)
    temp = temp.rename(index=str, columns={"VALUENUM": label})
    return temp

In [ ]:
def get_DataFrame(df):
    diabetes_df = getFeatures(df,211,'heart rate')
    diabetes_items = getItems(df)
    diabetes_items = diabetes_items[diabetes_items['ITEMID'].isin([646,52,618,113,8368,51])]
    for index, col in diabetes_items.iterrows():
        temp = getFeatures(df,col['ITEMID'],col['LABEL'])
        diabetes_df = pd.merge(diabetes_df, temp, how='outer', on=['SUBJECT_ID', 'ICUSTAY_ID','CHARTTIME'])
    return diabetes_df

In [ ]:
def getItems(df):
    items_df = pd.DataFrame(columns=['ITEMID', 'LABEL'])
    for x in [646,52,618,8368,51,497,678]:
        temp = D_ITEMS_df[D_ITEMS_df['ITEMID']==x]
        temp = temp.drop(temp.columns.difference(['ITEMID','LABEL']),axis=1)
        items_df = pd.concat([items_df,temp], ignore_index=True)
    return items_df

In [ ]:
print(datetime.datetime.now())

# items to be removed 
test_subject = {109, 188, 209, 249, 3, 6} 
diabetes_subjectID = [ele for ele in diabetes_subjectID if ele not in test_subject]

icu_1hour_df = pd.DataFrame()

for x in chartevents_all_df['SUBJECT_ID'].unique().tolist():
    temp = align_dataframe(x,chartevents_all_df)
    icu_1hour_df = icu_1hour_df.append(temp, sort = False)

print(datetime.datetime.now())